In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ephesus.sentence import *
import pandas as pd

In [4]:
data = extract_json("PROJECT_EPHESUS-labeling_V02.json")
dico = tune_data(data)
train_set, test_set = create_training_set(dico)

In [5]:
create_training_data(train_set, "train_test_02.spacy")

100%|████████████████████████████████████████████████████████████████████████████████| 360/360 [00:03<00:00, 117.85it/s]

training data created under '../raw_data/train_test_02.spacy'


In [5]:
nlp = load_model("../models/model_small/model-best")

In [6]:
test = test_set["annotations"][5]['text']
doc = nlp(test)
spacy.displacy.render(doc, style="ent",jupyter=True)

In [6]:
create_training_data(test_set, "test_set_02.spacy")

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 202.35it/s]

training data created under '../raw_data/test_set_02.spacy'


Test de return_label vs. original labelisation

In [7]:
return_label(sentence,nlp)

NameError: name 'sentence' is not defined

In [ ]:
df_test = pd.DataFrame(test_set["annotations"])
liste_test = df_test.iloc[0,1]

In [ ]:
sentence = df_test.iloc[0,0]
sentence

In [ ]:
for elm in liste_test:
    print(sentence[elm[0]:elm[1]])
    print(elm[2])

In [ ]:
def test(sentence, ents):
    liste = []
    for elm in ents:
        
        liste.append((sentence[elm[0]:elm[1]],elm[2]))
    return liste
        
        
    

In [ ]:
test(sentence,liste_test)

In [ ]:
create_training_data(test_set, "test_set.spacy")

# Tuto from https://www.kaggle.com/code/finalepoch/medical-ner-using-spacy

In [ ]:
from __future__ import unicode_literals, print_function
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import spacy
import sys

In [ ]:
dico

In [ ]:
def train_model(model=None, output_dir="../models/test_model", n_iter=1000):
    
    if model is not None:
        nlp = spacy.load(model)
        print("Loaded model '%s'" %model)
    else:
        nlp = spacy.blank("fr")
        print("Created blank 'fr' model")
        
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe('ner')
    else:
        ner = nlp.get_pipe("ner")
        
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(dico["annotations"])
            losses = {}
            batches = minibatch(dico["annotations"], size=compounding(4.0, 64.0, 1.2))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  
                    annotations,  
                    drop=0.20, 
                    losses=losses
                   
                )
            print("Losses", losses)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [ ]:
nlp = spacy.blank("fr")

In [ ]:
nlp.add_pipe('ner')

In [ ]:
nlp = load_model("../models/model_small/model-best")

In [ ]:
nlp.pipe_names

In [ ]:
TRAIN_DATA = dico["annotations"]

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

nlp = spacy.blank("fr")
nlp.add_pipe('ner')
nlp.begin_training()
# TRAINING THE MODEL  

In [ ]:
text = TRAIN_DATA[0]["text"]
text

In [ ]:
annotations = TRAIN_DATA[1]
annotations

In [ ]:
example = Example.from_dict(nlp.make_doc(text), annotations)

In [ ]:
example

In [ ]:
nlp.update([example])

In [ ]:
TRAIN_DATA

In [ ]:
TRAIN_DATA[0]

In [ ]:
{'entities':TRAIN_DATA[1]['entities']}

In [ ]:
len(TRAIN_DATA)

In [ ]:
nlp = spacy.blank("fr")
nlp.add_pipe('ner')
nlp.begin_training()
# TRAINING THE MODEL

example = []
i= 0
for elm in TRAIN_DATA:
    example = Example.from_dict(nlp.make_doc(elm['text']),{'entities':elm['entities']})
    nlp.update([example], drop=0.5)
    print("Losses", losses)
    print(i)
    i += 1
    
    

In [ ]:
TRAIN_DATA[9]

In [ ]:
doc = nlp("Sous-cutanée débuté le 3 septembre, 2 fois par jour. Jusqu'au. 8 septembre.")
spacy.displacy.render(doc, style="ent",jupyter=True)